In [78]:
import sys
sys.path.append('C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/backend/chatbot_utils')

import context_model as cm
import chatbot_utils as chatbot
import importlib
importlib.reload(cm)
importlib.reload(chatbot)

import pandas as pd
import hashlib
import json

with open('C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/openAI_api.json') as f:
    key = json.load(f)

DATA_DIR = 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset'


### Testing Embedding and Context Models

In [2]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()

In [76]:
cm.convert_logs_to_embeddings(f"{DATA_DIR}/final_log.out")

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-ada-002 in organization org-kfsNXpcw90CoawqSyD7Mw4CD on tokens per min (TPM): Limit 1000000, Used 785665, Requested 317726. Please try again in 6.203s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [13]:
returns = cm.get_context("When was run sfa_mmi executed?", f"./chromadb/{md5hash}_large")
returns

([Document(page_content='Nov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.verify_args:63]: sri_recall_log_lookup_location: /home/instrument/fw/log/2023-11-09T13-11-48Z/sfa/NetworkEmulation1/sri\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.verify_args:69]: working_dir: /home/instrument/fw/log/2023-11-09T13-11-48Z/sfa/NetworkEmulation1/sri\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.__init__:43]: SRSS version: 05.01.01\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,306+0000] [utilities.register_in_persistence_registry_service:215]: Successfully registered in PersistenceRegistryService\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,306+0000] [srss.__init__:51]: Service registered\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27255]: PyInstaller _MEI directory save_rec

### ChatGPT test

In [35]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()


In [36]:
chatbot.chat_with_bot("Were there any SSH login errors?", md5hash, '002', memory=False)

{'answer': 'Yes, there were SSH login errors. There were instances of authentication failure and failed passwords for invalid users. There were also errors due to the connection being closed by the remote host.\n',
 'Source': ['Nov 09 14:34:40 CMX50070-101776 sshd[538343]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=10.0.2.217\nNov 09 14:34:42 CMX50070-101776 sshd[538343]: Failed password for invalid user pi from 10.0.2.217 port 41984 ssh2',
  'Nov 09 14:34:38 CMX50070-101776 sshd[538103]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=10.0.2.217\nNov 09 14:34:39 CMX50070-101776 sshd[538103]: Failed password for invalid user USERID from 10.0.2.217 port 41888 ssh2',
  'Nov 09 13:59:25 CMX50070-101776 sshd[308115]: error: kex_exchange_identification: Connection closed by remote host\nNov 09 13:59:25 CMX50070-101776 sshd[308115]: Connection closed by 172.25.11.231 port 41772',
  'Nov 09 13:59:06 CMX50070-

In [37]:
chatbot.chat_with_bot("What IPs were they?", md5hash, '002', memory=True)

{'answer': 'The IP was 10.0.2.217.\n',
 'Source': ['Nov 09 14:28:28 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217\nNov 09 14:28:28 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217',
  'Nov 09 14:34:11 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217\nNov 09 14:34:19 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217',
  'Nov 09 14:31:45 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217\nNov 09 14:31:45 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217',
  'Nov 09 14:31:45 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217\nNov 09 14:31:45 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217',
  'Nov 09 14:31:45 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217\nNov 09 14:31:4

In [38]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory

import os
os.environ["OPENAI_API_KEY"] = key['openai_api_key']

In [39]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, input_key='question', output_key='answer')

In [40]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()
question = "What lines have messages on SSH?"

In [41]:
docs_small,conf_small = cm.get_context(question, f"./chromadb/{md5hash}_small")
docs_large,conf_large = cm.get_context(question, f"./chromadb/{md5hash}_large")

## Add both large and small to a single list and sort them by confidence
docs = docs_small + docs_large
conf = conf_small + conf_large

df = pd.DataFrame({'docs':docs, 'conf':conf})
df.sort_values(by='conf', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

# df



In [22]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)

vectorstore_relevant = Chroma.from_documents(list(df['docs'].values), embedding=embedding)
retriver = vectorstore_relevant.as_retriever(search_kwargs={"k": 10,"score_threshold": .5})


In [23]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
max_tokens=1000,
)

In [24]:
chat = RetrievalQAWithSourcesChain.from_llm(llm, retriever=retriver, memory=memory, return_source_documents=True, max_tokens_limit=1000, reduce_k_below_max_tokens=True)

In [25]:
result = chat({"question": question})
result

{'question': 'What lines have messages on SSH?',
 'chat_history': [HumanMessage(content='What lines have messages on SSH?'),
  AIMessage(content='The following lines have messages on SSH: \n\n1. "Nov 09 13:50:04 CMX50070-101776 sshd[249289]: Connection closed by 172.25.11.232 port 45426"\n2. "Nov 09 13:44:11 CMX50070-101776 sshd[212198]: Connection closed by 172.25.232.145 port 56312"\n3. "Nov 09 13:54:33 CMX50070-101776 sshd[277530]: Connection closed by 172.25.11.232 port 36124"\n4. "Nov 09 13:55:08 CMX50070-101776 sshd[281144]: Connection closed by 192.168.85.173 port 60810"\n5. "Nov 09 13:53:34 CMX50070-101776 sshd[271352]: Connection closed by 172.24.11.231 port 33828"\n\n')],
 'answer': 'The following lines have messages on SSH: \n\n1. "Nov 09 13:50:04 CMX50070-101776 sshd[249289]: Connection closed by 172.25.11.232 port 45426"\n2. "Nov 09 13:44:11 CMX50070-101776 sshd[212198]: Connection closed by 172.25.232.145 port 56312"\n3. "Nov 09 13:54:33 CMX50070-101776 sshd[277530]: Conn

In [26]:
print(result['answer'])

The following lines have messages on SSH: 

1. "Nov 09 13:50:04 CMX50070-101776 sshd[249289]: Connection closed by 172.25.11.232 port 45426"
2. "Nov 09 13:44:11 CMX50070-101776 sshd[212198]: Connection closed by 172.25.232.145 port 56312"
3. "Nov 09 13:54:33 CMX50070-101776 sshd[277530]: Connection closed by 172.25.11.232 port 36124"
4. "Nov 09 13:55:08 CMX50070-101776 sshd[281144]: Connection closed by 192.168.85.173 port 60810"
5. "Nov 09 13:53:34 CMX50070-101776 sshd[271352]: Connection closed by 172.24.11.231 port 33828"




### Full Token Test

In [49]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, input_key='question', output_key='answer')

In [50]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)

# Loading the datastore
vector_db = Chroma(persist_directory=f"./chromadb/{md5hash}_small", embedding_function=embedding)
retriver = vector_db.as_retriever()

In [51]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
)

chat = RetrievalQAWithSourcesChain.from_llm(llm, retriever=retriver, memory=memory, return_source_documents=True)

In [53]:
result = chat({"question": f"What IPs were they from?"})
result

{'question': 'What IPs were they from?',
 'chat_history': [HumanMessage(content='Were there any SSH login errors?'),
  AIMessage(content='Yes, there were SSH login errors. These include authentication failures and connection closures by the remote host. \n'),
  HumanMessage(content='What IPs were they from?'),
  AIMessage(content='The IP they were from is 10.0.2.217.\n')],
 'answer': 'The IP they were from is 10.0.2.217.\n',
 'sources': 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset/test_log1.out',
 'source_documents': [Document(page_content='Nov 09 14:28:28 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217\nNov 09 14:28:28 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217', metadata={'source': 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset/test_log1.out'}),
  Document(page_content='Nov 09 14:31:45 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0

### Langchain Summary Model

In [69]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory
import tiktoken

import os
os.environ["OPENAI_API_KEY"] = key['openai_api_key']

md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()


In [110]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
import textwrap
from time import monotonic

prompt_template = """Write a concise summary of the following. Start with a small paragraph and then mention some key events as bullet points:


{text}

Paragraph:

Important points:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

In [111]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
)

In [113]:
# loader = TextLoader(f"{DATA_DIR}/test_log1.out")
# documents = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# documents = text_splitter.split_documents(documents)
question = 'What are errors and important processes in the log?'
docs_small,conf_small = cm.get_context(question, f"./chromadb/{md5hash}_small", k = 30)
# docs_large,conf_large = cm.get_context(question, f"./chromadb/{md5hash}_large", k = 30)

## Add both large and small to a single list and sort them by confidence
docs = docs_small

In [114]:
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
summary = chain.run(docs)

In [115]:
print(summary)

The logs show a series of operations on the CMX50070-101776 server. There are recurring error messages indicating that no loggers are registered in the RsLicenseServerApp and the AlfManagerApp. However, the server continues to perform a variety of tasks, including starting applications, setting operational modes, and rotating and compressing logs.

Key events:

- Multiple instances of "No Loggers registered!" error in RsLicenseServerApp and AlfManagerApp.
- Starting the EventLog with 19 arguments, reading configurations from '/usr/lib/eventlog/eventlog.ini', and disabling logging to stdout.
- Logs being compressed under "/home/instrument/fw/log/sysmond_processes.1.nmon".
- Daily rotation of log files, cleanup of temporary directories, and smartmontools data collection being triggered by the system.
- The Xserver using the config directory: "/etc/X11/xorg.conf.d".
- AlfManager started with version: 1.9.4.239 and operation mode set to 'Primary'.
- Detected change of CLOCK_REALTIME in the

## Clustering attempt

In [117]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

In [99]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)
vector_db = Chroma(persist_directory=f"./chromadb/{md5hash}_small", embedding_function=embedding)


In [140]:
embeddings = np.array(vector_db.get(include=['embeddings'])['embeddings'])

pca = PCA(n_components=10)
pca.fit(embeddings)

pca_embeddings = pca.transform(embeddings)

## DBSCAN
dbscan = DBSCAN(eps=0.05, min_samples=10)
dbscan.fit(pca_embeddings)

## Get all the core points and outliers
core_points = np.where(dbscan.core_sample_indices_)[0]
outliers = np.where(dbscan.labels_ == -1)[0]

## Get the unique labels
labels = np.unique(dbscan.labels_)
labels

In [137]:
outliers.shape

(2287,)

In [138]:
core_points.shape

(13495,)